In [1]:
import pandas as pd
import tensorflow as tf
import keras
from keras.applications import ResNet50, ResNet101, ResNet152
from tensorflow.keras.utils import to_categorical
from torchvision import transforms
from PIL import Image
import torch

def print_model_acc(target_col):
    image_tensors = None
    image_tensors2 = None

    # 데이터프레임 가져오기
    train_df = pd.read_csv('./Data/train_df.csv', index_col=0)
    train_df = train_df[train_df['image_path'].str.contains(target_col)]
    train_df = train_df.reset_index(drop=True)

    test_df = pd.read_csv('./Data/test_df.csv', index_col=0)
    test_df = test_df[test_df['image_path'].str.contains(target_col)]
    test_df = test_df.reset_index(drop=True)

    # 이미지 파일 텐서로 변환 - train 데이터
    transform = transforms.Compose([
        transforms.Resize((128, 128)),  # 이미지 크기 조정
        transforms.ToTensor(),  
        transforms.Normalize(mean=[0.25, 0.25, 0.25], std=[0.1, 0.1, 0.1]) # 이미지 정규화
    ])

    tensor_list = []
    for image_path in train_df['image_path']:
        image = Image.open(image_path)
        tensor = transform(image)
        tensor = tensor.transpose(0,2) # shape 순서 변경(추후 텐서플로로 변경하기위해)
        tensor_list.append(tensor)

    image_tensors = torch.stack(tensor_list) #train데이터
    print('이미지 파일 텐서로 변환 - train 데이터 완료')
    
    # 이미지 파일 텐서로 변환 - test 데이터
    tensor_list2 = []
    for image_path in test_df['image_path']:
        image2 = Image.open(image_path)
        tensor2 = transform(image2)
        tensor2 = tensor2.transpose(0,2)
        tensor_list2.append(tensor2)

    image_tensors2 = torch.stack(tensor_list2) #test데이터

    # tensorflow로 변환
    X_train = tf.convert_to_tensor(image_tensors.numpy())
    y_train = train_df[target_col].values

    X_test = tf.convert_to_tensor(image_tensors2.numpy())
    y_test = test_df[target_col].values
    print('이미지 파일 텐서로 변환 - test 데이터 완료')

    # 증상 진행 단계 원핫인코딩
    y_train_oh = to_categorical(y_train)
    y_test_oh = to_categorical(y_test)

    model = ResNet50(weights=None, include_top=True, input_shape=(128, 128, 3), pooling=max, classes=4)

    sgd = keras.optimizers.SGD(learning_rate=0.01, decay= 1e-6 ,momentum=0.9, nesterov=True) 

    model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer= sgd) 
    model.fit(X_train, y_train_oh, batch_size=16, epochs=1)

    test_loss, test_acc = model.evaluate(X_test, y_test_oh, verbose=2)
    print(target_col +' test_loss: ' ,test_loss)
    print(target_col +' test_acc: ' ,test_acc)

In [10]:
print_model_acc('탈모')

이미지 파일 텐서로 변환 - train 데이터 완료
이미지 파일 텐서로 변환 - test 데이터 완료
290/290 [==============================] - 8421s 29s/step - loss: 1.1024 - accuracy: 0.7262
166/166 - 582s - loss: 0.6952 - accuracy: 0.7352 - 582s/epoch - 4s/step
탈모 test_loss:  0.6951689720153809
탈모 test_acc:  0.7352496385574341


In [2]:
print_model_acc('탈모')

이미지 파일 텐서로 변환 - train 데이터 완료
이미지 파일 텐서로 변환 - test 데이터 완료
1158/1158 [==============================] - 8929s 8s/step - loss: 0.9382 - accuracy: 0.7133
166/166 - 585s - loss: 0.6834 - accuracy: 0.7396 - 585s/epoch - 4s/step
탈모 test_loss:  0.683415949344635
탈모 test_acc:  0.7395991086959839
